## 多元线性回归
在简单线性回归中，用的样本特征值只有一个，但是在真实的世界中，我们的样本有N个特征值，针对这样的样本，我们依然可以使用线性回归的思维来解决。

![](img/isea29.jpg)
此时大X 是一个矩阵，Xi表示第i个特征，在简单线性回归中，我们需要求出两个参数，在多元线性回归中，我们需要求出 N+1 个参数，也即找到 N + 1 个 θ 的值，来使得误差（这里是平方损失函数）尽可能的小。

这里的$\theta_0$是截距，对于多元线性回归来说需要求出N + 1 个参数。将上面的公式化简为向量相乘的形式：
![](img/isea30.jpg)
这里的$\theta$表示为列向量的形式，并且给$\theta_0$虚构第0个特征$X^{(i)}_0$,且$X^{(i)}_0 \equiv 1$ (实际上我们添加了一个虚拟的特征，且全部的特征值都是1)

上面的$X^{(i)}$ 是一个行向量，$X$ 是一个矩阵。那么此时$\hat{y}^{(i)} = X^{(i)} \cdot \theta$ (第i个元素的估计 等于第i个样本 点乘 $\theta$

![](img/isea31.jpg)
此时，我们得到y的估计是一个列向量

对于上述的公式，我们可以使用向量化运算：**相乘再相加 基本都可以转为向量运算**
![](img/isea32.jpg)

所以目标即使估计一个向量θ是的上式尽可能的小，使用最小二乘法（对$\theta$中的每一个变量求导然后令其等于0）来求解该公式的时候（还有另外的解法，不需要推导），该式的解为：
![](img/isea33.jpg)

这里得到的是数学解，对于KNN算法，在处理之前，我们需要对数据进行归一化的处理，但是对于多元线性回归问题，我们不需要对数据进行归一化处理，因为这里得到的θ是每一个特征前面的系数，不存在量纲的问题。

![](img/isea34.jpg)

将θ分开的原因是，系数部分都对应着原来样本中的每一个特征，该系数可以描述特征对于最终样本影响的程度，而$\theta_0$是一个偏移。

在sk-learn中也对系数进行了拆分

#### 在波士顿房产例子中，考虑所有的特征

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [18]:
boston = datasets.load_boston()
X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]
X.shape

(490, 13)

In [4]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 6666)

In [22]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
# 这里是多个特征，所以是多元线性回归
lin_reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [20]:
# θ的值
lin_reg.coef_

array([-7.15800921e-02,  3.40765969e-02, -6.04949819e-02,  3.67953369e-01,
       -1.14244082e+01,  4.10778262e+00, -2.89300131e-02, -1.14261161e+00,
        2.24747826e-01, -1.39900626e-02, -7.84454442e-01,  7.91217343e-03,
       -2.81714455e-01])

In [7]:
# 截距
lin_reg.intercept_

27.908790132386766

In [21]:
# 计算R Squared的值来判断多元线性回归预测的优劣
lin_reg.score(X_test,y_test)

0.753170345928847

### 使用KNN Regressor来解决回归问题(regressor回归器的意思)

In [9]:
from sklearn.neighbors import KNeighborsRegressor

knn_reg = KNeighborsRegressor()  # 默认的k 取值为5
knn_reg.fit(X_train,y_train)
knn_reg.score(X_test,y_test)

0.5233373584627842

#### 使用网格搜索找到最好的参数

In [26]:
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
standardScaler.fit(X_train, y_train)
X_train_standard = standardScaler.transform(X_train)
X_test_standard = standardScaler.transform(X_test)


from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        "weights": ["uniform"],
        "n_neighbors": [i for i in range(1, 11)]
    },
    {
        "weights": ["distance"],
        "n_neighbors": [i for i in range(1, 11)],
        "p": [i for i in range(1,6)]
    }
]

knn_reg = KNeighborsRegressor()
grid_search = GridSearchCV(knn_reg, param_grid, n_jobs=-1, verbose=1)
grid_search.fit(X_train_standard, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   11.7s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [27]:
grid_search.best_params_

{'n_neighbors': 4, 'p': 2, 'weights': 'distance'}

In [28]:
# 交叉验证之后的评分
grid_search.best_score_

0.7916530315160019

In [30]:
# R方的评分  和前面的多元线性回归的评分使用的是同样的一个标准
grid_search.best_estimator_.score(X_test_standard, y_test)

0.7857965332185303

**所以在评测某个机器学习的方法的好坏的时候，需要使用同一个标准才行**

### 关于线性回归的可解释性

lin_reg.intercept_ 中存放的是特征的系数，正数表示正相关，负数表示负相关，绝对值越大，表示的影响越大。在本例子中是特征对于房价的影响

In [10]:
lin_reg.coef_

array([-7.15800921e-02,  3.40765969e-02, -6.04949819e-02,  3.67953369e-01,
       -1.14244082e+01,  4.10778262e+00, -2.89300131e-02, -1.14261161e+00,
        2.24747826e-01, -1.39900626e-02, -7.84454442e-01,  7.91217343e-03,
       -2.81714455e-01])

In [11]:
np.argsort(lin_reg.coef_) # 从小到大排列

array([ 4,  7, 10, 12,  0,  2,  6,  9, 11,  1,  8,  3,  5], dtype=int64)

In [12]:
boston.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [13]:
# 查看哪些特征对放假的影响是最大的
boston.feature_names[np.argsort(lin_reg.coef_)]

array(['NOX', 'DIS', 'PTRATIO', 'LSTAT', 'CRIM', 'INDUS', 'AGE', 'TAX',
       'B', 'ZN', 'RAD', 'CHAS', 'RM'], dtype='<U7')

In [32]:
# 查看这些特征的具体含义
print(boston.DESCR)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

![](img/isea35.jpg)

使用R squared来衡量我们的模型的优劣。

* 典型的参数学习，对比KNN是非参数学习；
* 只能解决回归问题，虽然很多分类方法中，线性回归是基础（如逻辑回归）；对比KNN，既可以解决分类问题，也可以解决回归问题。
* 在使用线性回归的时候，是有一个假设，也即数据满足一定的线性关系，KNN对数据没有假设，但当我们的输入和输出有较强的线性关系的时候，线性回归算法是比KNN这个算法效果要好
* 优点：对数据有较强的解释性。 

**在使用线性回归的时候，有一个假设：数据之间存在线性关系**

上面的求解目标函数的时候，我们使用的是正规的方程解，这样方式复杂度较高，还有另外的解法：梯度下降法，该方法是目标函数的通用解法